**Group 007-13 Project Proposal**

**Introduction**
The data set we chose is a tennis data set. We chose to keep specific columns in order to remove irrelevant data and keep the most important columns. We decided to keep if they are right or left handed, their current ranking, the player's name, their backhand throw, their current elo rank, and their prize money. A player's current elo rank is an estimate of their strength and is affected by their wins and loses. We are wondering whether a players bankhand throw and whether they are left or right handed affects their overall success. We will evaluate this by using their backhand, handedness, current rank, and current elo rank. 

**Preliminary exploratory data analysis**
*In code cells below, demonstrate that data can be read into R and then follow the instructions on Canvas*

In [12]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

download.file("https://drive.google.com/uc?export=download&id=1fOQ8sy_qMkQiQEAO6uFdRX4tLI8EpSTn", "tennis_data.csv")
tennis_data <- read_csv("tennis_data.csv", show_col_types = FALSE) |> 
    mutate(play_surface = as_factor(surface), tourney_name = as_factor(tourney_name), winner_hand = as_factor(winner_hand), loser_hand = as_factor(loser_hand)) |>
    select(tourney_name, play_surface, winner_hand, winner_age, loser_hand, loser_age, minutes, winner_rank, loser_rank)
tennis_data

tennis_split <- initial_split(tennis_data, prop = .75, strata = )  
tennis_train <- training(tennis_split)   
tennis_test <- testing(tennis_split)
tennis_train
tennis_test

New names:
• `` -> `...1`


tourney_name,play_surface,winner_hand,winner_age,loser_hand,loser_age,minutes,winner_rank,loser_rank
<fct>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
Brisbane,Hard,R,29.00479,R,22.88569,124,9,16
Brisbane,Hard,R,22.88569,R,33.70568,82,16,239
Brisbane,Hard,R,29.00479,R,31.88227,66,9,40
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Davis Cup WG F: BEL vs FRA,Hard,R,32.59411,R,33.68925,106,15,76
Davis Cup WG F: BEL vs FRA,Hard,R,26.95414,R,32.59411,164,7,15
Davis Cup WG F: BEL vs FRA,Hard,R,23.73990,R,33.68925,94,18,76


tourney_name,play_surface,winner_hand,winner_age,loser_hand,loser_age,minutes,winner_rank,loser_rank
<fct>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
Antalya,Grass,L,29.98768,R,29.23751,103,24,47
Australian Open,Hard,R,21.65914,R,30.65298,201,58,14
Montpellier,Hard,R,28.64613,R,29.38261,124,109,41
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Newport,Grass,R,27.98084,R,32.85147,136,72,107
Davis Cup Finals QF: GBR vs GER,Hard,R,29.48939,R,29.56605,141,42,35
Delray Beach,Hard,R,21.98220,R,26.81451,96,98,162


tourney_name,play_surface,winner_hand,winner_age,loser_hand,loser_age,minutes,winner_rank,loser_rank
<fct>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
Brisbane,Hard,R,33.70568,R,19.86858,106,239,31
Brisbane,Hard,R,22.88569,R,28.01095,129,16,18
Brisbane,Hard,R,29.00479,R,27.62765,100,9,19
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Tour Finals,Hard,R,26.93498,R,36.26557,105,8,2
Tour Finals,Hard,R,26.49692,R,26.93498,150,6,8
Davis Cup WG F: BEL vs FRA,Hard,R,32.59411,R,33.68925,106,15,76


**Methods**
*Explain how you will conduct either your data analysis and which variables/columns you will use.
Describe at least one way that you will visualize the results*

**Expected outcomes and significance** *Answer questions on Canvas*